# Imports

In [0]:
import torch

In [0]:
from scipy import io
from PIL import Image
import numpy as np
import os

In [0]:
# change path to the PASCAL folder here
PATH = 'PASCAL3D+_release1.1/'

# change path to where you want the resulting numpy arrays to be saved
RESULT_PATH = 'Data/numpy_data/'

# helper functions 
- get object ids within an image
- get true rotations
- get bounding box


In [0]:
def check_image_usable_and_get_objectid_from_annotations(annotations_mat, cls):
  infos = []

  objectid = -1
  for i in range(annotations_mat['record'][0][0]['objects'].shape[1]):
    print(i)
    if annotations_mat['record'][0][0]['objects'][0][i]['class'][0] == cls:

      objectid = i
      shape_idx = annotations_mat['record'][0][0]['objects'][0][0]['cad_index'][0][0] - 1
      occluded = 0
      if annotations_mat['record'][0][0]['objects'][0][objectid]['occluded'][0][0] == 1:
        print('occluded')
        occluded = 1
      truncated = 0
      if annotations_mat['record'][0][0]['objects'][0][objectid]['truncated'][0][0] == 1:
        print('truncated')
        truncated = 1
      infos.append((objectid, shape_idx, occluded, truncated))

  return infos


def create_rotation_from_aet(rot_z, rot_x, rot_y):
  #from azimuth, elevation, tilt

  rot_matrix_z = np.array([[np.cos(rot_z), -np.sin(rot_z), 0.],
                                      [np.sin(rot_z), np.cos(rot_z), 0.],
                                      [0., 0., 1.]])

  rot_matrix_x = np.array([[1., 0., 0.],
                                      [0., np.cos(rot_x), -np.sin(rot_x)],
                                      [0., np.sin(rot_x), np.cos(rot_x)]])

  rot_matrix_y = np.array([[np.cos(rot_y), 0., -np.sin(rot_y)],
                           [0.,1.,0.],
                           [np.sin(rot_y), 0., np.cos(rot_y)]])

  return np.matmul(np.matmul(rot_matrix_z, rot_matrix_x), rot_matrix_y)


def get_true_rot_from_annotations(annotations_mat, objectid):
  annotations = annotations_mat['record'][0][0]['objects'][0][objectid]['viewpoint'][0][0]

  rot = create_rotation_from_aet(np.deg2rad(annotations['azimuth'][0][0]), -np.deg2rad(annotations['elevation'][0][0]), -np.deg2rad(annotations['theta'][0][0]))
  return torch.FloatTensor(rot)

def get_bbox_from_annotations(annotations_mat, objectid):
  bbox = annotations_mat['record'][0][0]['objects'][0][objectid]['bbox'][0]
  return np.array(bbox, dtype=np.float16)

# main bit
load image sets
and from every id, find all objects in an image& save the corresponding info in numpy arrays

In [0]:
classes = ['aeroplane', 'bicycle', 'boat', 'bottle', 'bus', 'car', 'chair', 'diningtable', 'motorbike', 'sofa', 'train', 'tvmonitor']

# make directories:
for cls in classes:
  mypath = RESULT_PATH + 'Train/' + cls
  !mkdir $mypath
  mypath = RESULT_PATH + 'Test/' + cls
  !mkdir $mypath


shapes_per_class = {'aeroplane':8,
                    'bicycle':6,
                    'boat':6,
                    'bottle':8,
                    'bus':6,
                    'car':10,
                    'chair':10,
                    'diningtable':6,
                    'motorbike':5,
                    'sofa':6,
                    'train':4,
                    'tvmonitor':4}


for split in ['train', 'val']:

  for cls in classes:
      file_val = open(PATH + "Image_sets/{}_imagenet_{}.txt".format(cls, split), 'r')
      all_im_ids = file_val.read().split()
      print(len(all_im_ids))
      print(all_im_ids[1])

      images = []
      valid_im_ids = []
      shape_idxs = []
      true_rots = []
      bboxes = []
      occludeds = []
      truncateds = []

      i = 0
      for j in range(0, len(all_im_ids)):
        im_id = all_im_ids[j]
        annotations_mat = io.loadmat(PATH + 'Annotations/' + cls + '_imagenet/' +im_id+ ".mat")
        infos = check_image_usable_and_get_objectid_from_annotations(annotations_mat, cls)
        for (index, shape, occluded, truncated) in infos:
          if index != -1:
            with Image.open('Images/' + cls + '_imagenet/' + im_id + '.JPEG') as im:
              images.append(np.array(im))
            if shape >= shapes_per_class[cls]:
              print('ERROR..', im_id)

            shape_idxs.append(shape)
            valid_im_ids.append(im_id)
            true_rots.append(get_true_rot_from_annotations(annotations_mat, index))
            bboxes.append(get_bbox_from_annotations(annotations_mat, index))
            occludeds.append(occluded)
            truncateds.append(truncated)
          i += 1


      print(len(valid_im_ids))
      print(len(shape_idxs))
      print(len(true_rots))
      print(len(bboxes))
      print(len(occludeds))
      print(len(truncateds))
      print('\n\n')
      
      
      valid_im_ids = np.array(valid_im_ids)
      shape_idxs = np.array(shape_idxs)
      true_rots = np.array([np.array(x) for x in true_rots])
      bboxes = np.array(bboxes)
      images = np.array(images)
      occludeds = np.array(occludeds)
      truncateds = np.array(truncateds)
      
      print('image dim smaller than 3:')
      i = 0
      while i < len(images):
        im = images[i]
        if im.ndim < 3:
          print(i)
          images = np.delete(images, i, 0)
          valid_im_ids = np.delete(valid_im_ids, i, 0)
          true_rots = np.delete(true_rots, i, 0)
          shape_idxs = np.delete(shape_idxs, i, 0)
          bboxes = np.delete(bboxes, i, 0)
          occludeds = np.delete(occludeds, i, 0)
          truncateds = np.delete(truncateds, i, 0)
        else:
          i += 1
        

      print(valid_im_ids.shape)
      print(shape_idxs.shape)
      print(true_rots.shape)
      print(bboxes.shape)
      print(occludeds.shape)
      print(truncateds.shape)
      print('\n\n')
      print("saving...")

      if split == 'train':
        full_result_path = RESULT_PATH + 'Train/' + cls
      else:
        full_result_path = RESULT_PATH + 'Test/' + cls

      np.save(full_result_path + '/valid_im_ids', valid_im_ids)
      np.save(full_result_path + '/shape_idxs', shape_idxs)
      np.save(full_result_path + '/true_rots', true_rots)
      np.save(full_result_path + '/bboxes', bboxes)
      np.save(full_result_path + '/images', images, allow_pickle=True)
      np.save(full_result_path + '/occluded', occludeds)
      np.save(full_result_path + '/truncated', truncateds)
      
      print(cls, 'saved \n\n\n')